In [1]:
import numpy as np
import pandas as pd

from ipywidgets import interact
from ipywidgets.widgets import FloatSlider, IntSlider, Dropdown
import matplotlib.pyplot as plt

plt.ion()

In [2]:
def calculate_S1(row):
    if row.index == 0:
        return t1_init
    else:
        return 0
    
def calculate_Q1(row):
    q = a1 * (row['S1 (mm)'] - h1)**m1
    if q > 0:
        return q
    else:
        return 0

def initialize_df(df):
    # initialize the columns
    df['S1 (mm)'] = np.nan
    df.loc[0, 'S1 (mm)'] = t1_init
    df['Q1 (mm/h)'] = np.nan
    df['Q12 (mm/h)'] = np.nan
    df['S2 (mm)'] = np.nan
    df.loc[0, 'S2 (mm)'] = t2_init
    df['Q2 (mm/h)'] = np.nan
    df['Q1+Q2 (mm/h)'] = np.nan
    return df


### Interactive Inputs

In [16]:
def update_df(a1, m1, h1, a12, m12, h12, a2, m2, h2, t1_init, t2_init):
    df = initialize_df(pd.read_csv('data/data.csv', parse_dates=True, infer_datetime_format=True))
    for i in range(len(df)):
        if i == 0:
            this_s1 = t1_init
            this_s2 = t2_init

        else:
            q1_balance = (df.loc[i, 'Rain (mm/h)'] - df.loc[i, 'Evap (mm/h)'] - df.loc[i - 1, 'Q1 (mm/h)'] - df.loc[i - 1, 'Q12 (mm/h)'])/3
            this_s1 = max(0, df.loc[i - 1, 'S1 (mm)'] + q1_balance)

            delta_q2 = df.loc[i - 1, 'S2 (mm)'] + (df.loc[i - 1, 'Q12 (mm/h)'] - df.loc[i - 1, 'Q2 (mm/h)'])/3
            this_s2 = max(0, delta_q2)

        df.loc[i, 'S1 (mm)'] = this_s1
        df.loc[i, 'S2 (mm)'] = this_s2

        if this_s1 > h1:
            df.loc[i, 'Q1 (mm/h)'] = a1 * (this_s1 - h1)**m1
        else:
            df.loc[i, 'Q1 (mm/h)'] = 0

        if this_s1 > h12:
            df.loc[i, 'Q12 (mm/h)'] = a12 * (this_s1 - h12)**m12
        else:
            df.loc[i, 'Q12 (mm/h)'] = 0

        if this_s1 > h2:
            df.loc[i, 'Q2 (mm/h)'] = a2 * (this_s2 - h2)**m2

        df.loc[i, 'Q1+Q2 (mm/h)'] = df.loc[i, 'Q1 (mm/h)'] + df.loc[i, 'Q2 (mm/h)']
    return df

In [21]:
def plot_model(a1, m1, h1, a12, m12, h12, a2, m2, h2, t1_init, t2_init):
    df = update_df(a1, m1, h1, a12, m12, h12, a2, m2, h2, t1_init, t2_init)
    fig, ax = plt.subplots(2, 1, figsize=(16,6), sharex=True)
    df.plot('Date', 'Runoff (mm/h)', label='Observed', ax=ax[0], title='Observed vs. Simulated Runoff')
    df.plot('Date', 'Q1+Q2 (mm/h)', label='Simulated', ax=ax[0])
    df.plot('Date', 'Q1 (mm/h)', label='Tank 1', ax=ax[1])
    df.plot('Date', 'Q2 (mm/h)', label='Tank 2', ax=ax[1])
    ax[0].set_ylabel('Runoff (mm/h)')
    ax[1].set_ylabel('Runoff (mm/h)')
    plt.show()
#     return df

In [22]:
p = interact(plot_model, 
             a1=FloatSlider(min=0., max=2., step=0.01, value=0.02, continuous_update=False), 
             m1=FloatSlider(min=0., max=2., step=0.01, value=1.5, continuous_update=False),
             h1=FloatSlider(min=0., max=2., step=0.01, value=1.8, continuous_update=False),
             a12=FloatSlider(min=0., max=2., step=0.01, value=0.021, continuous_update=False),
             m12=FloatSlider(min=0., max=2., step=0.01, value=1., continuous_update=False),
             h12=FloatSlider(min=0., max=2., step=0.01, value=0., continuous_update=False),
             a2=FloatSlider(min=0., max=2., step=0.01, value=0.02, continuous_update=False),
             m2=FloatSlider(min=0., max=2., step=0.01, value=1.3, continuous_update=False),
             h2=FloatSlider(min=0., max=2., step=0.01, value=0., continuous_update=False),
             t1_init=FloatSlider(min=0., max=2., step=0.01, value=1.0, continuous_update=False),
             t2_init=FloatSlider(min=0., max=2., step=0.01, value=1.0, continuous_update=False),
            )
# p = interact(plot_model, a1=FloatSlider(0, 2), m1=FloatSlider(0, 2))

interactive(children=(FloatSlider(value=0.02, continuous_update=False, description='a1', max=2.0, step=0.01), …